<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>



<br>

# <font color="#76b900">**Notebook 7:** 使用向量存儲庫(Vector Store)的檢索增強生成(Retrieval-Augmented Generation)</font>

<br>

在前一個 notebook 中，我們學習了內嵌模型(Embedding Models)並練習了它們的一些功能。我們討論了它們在長篇的(long-form)文件比較的預期使用案例，並找到了將其作為自訂語義比較(custom semantic comparisons)骨幹的方法。這個 notebook 將進一步推進將這些想法朝向檢索(Retrieval)模型的預期使用案例，並探索如何建構依賴*向量存儲庫(Vector Store)*來自動保存和檢索(Retrieval)資訊的聊天機器人系統。


<br>

### **學習目標：**

-   了解語義相似性支援的系統如何促進易於使用的檢索(Retrieval)制定。
-   學習如何將檢索(Retrieval)模組整合到您的聊天模型系統中，用於檢索增強生成(Retrieval-Augmented Generation, RAG)管線(Pipeline)，這可以應用於文件檢索(Retrieval)和對話記憶(conversation memory)緩衝區等任務。

<br>

### **值得思考的問題：**


-   這個 notebook 不嘗試整合階層結構(Hierarchy)分析推理(Reasoning)或實用的(non-naive) RAG（如規劃 Agent）。考慮需要什麼修改才能使這些組件在 LCEL 鏈(Chain)中運作。
-   考慮何時最好將您的向量存儲庫(Vector Store)解決方案移至可擴展服務，以及何時 GPU 對於最佳化變得必要。

<br>

### **環境設置：**

In [1]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")



<br>

## **第一部分：** RAG 工作流程摘要


這個 notebook 將探索幾個作法(Paradigm)並推導參考程式碼，幫助您處理一些最常見(popular)的檢索增強(Retrieval-Augmented)工作流程。具體來說，將涵蓋以下部分（並突出差異）：

> ***對話交流的向量存儲庫(Vector Store)工作流程：***

-   為每個新對話生成語義內嵌(Embedding)。

-   將訊息主體添加到向量存儲庫(Vector Store)以供檢索(Retrieval)。

-   查詢向量存儲庫(Vector Store)以獲取相關訊息來填充 LLM 脈絡資訊(Context)。

> ***任意文件的修改工作流程：***

-   **將文件分割成分塊(Chunking)並處理成有用的訊息。**

-   為每個**新文件分塊**生成語義內嵌(Embedding)。

-   將**分塊主體**添加到向量存儲庫(Vector Store)以供檢索(Retrieval)。

-   查詢向量存儲庫(Vector Store)以獲取相關**分塊**來填充 LLM 脈絡資訊(Context)。

    -   ***可選：* 修改/綜合結果(Synthesis)以獲得更好的 LLM 結果。**

> **任意文件目錄的擴展工作流程：**

-   將**每個文件**分割成分塊(Chunking)並處理成有用的訊息。

-   為每個新文件分塊生成語義內嵌(Embedding)。

-   將分塊主體添加到 **可擴展的向量資料庫以進行快速檢索(Retrieval)** 。

    -   ***可選：* 利用階層結構(Hierarchy)或中介資料(Metadata)結構來處理更大的系統。**

-   查詢**向量資料庫**以獲取相關分塊來填充 LLM 脈絡資訊(Context)。

    -   *可選：* 修改/綜合結果(Synthesis)以獲得更好的 LLM 結果。

圍繞 RAG 的一些最重要術語在 [**LlamaIndex 概念頁面**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html) 中有詳細介紹，這本身就是針對 LlamaIndex 載入和檢索(Retrieval)策略進展的絕佳起點。我們強烈建議您在繼續這個 notebook 時將其作為參考，並建議您在課程結束後嘗試 LlamaIndex，以親身考慮其優缺點！

<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> 來自  [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/)





<br>

## **第二部分：** 對話歷史紀錄(History)的 RAG


在我們之前的探索中，我們深入研究了文件內嵌模型(Embedding Models)的功能，並使用它們來內嵌(Embedding)、儲存和比較文字(Text)的語義向量表示。雖然我們可以思考如何手動有效地將其擴展到向量存儲庫(Vector Store)領域，但使用標準 API 的真正美妙之處在於它與其他已經可以為我們做繁重工作的框架的強力整合！

### **步驟 1**：獲得對話


考慮一個使用 Llama-13B 在聊天 Agent 和名為 Beras 的藍熊之間製作的對話。這個對話充滿了細節和潛在的分歧，為我們的研究提供了豐富的資料集：

In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]


使用前一個 notebook 的手動內嵌(Embedding)策略仍然非常可行，但我們也可以放心讓**向量存儲庫(Vector Store)**為我們完成所有工作！

### **步驟 2：** 建構我們的向量存儲庫(Vector Store)檢索器


為了簡化對我們對話的相似性查詢，我們可以使用向量存儲庫(Vector Store)來幫助我們追蹤段落！ **向量存儲庫(Vector Store)** ，或向量儲存系統，抽象化了內嵌(Embedding)/比較策略的大部分低階細節，並提供了一個簡單的介面來載入和比較向量。

<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> 來自 [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)



<br>


除了從 API 角度簡化流程外，向量存儲庫(Vector Store)還在底層實作連接器(Connector)、整合和最佳化。在我們的情況下，我們將從 [**FAISS 向量存儲庫(Vector Store)**](https://python.langchain.com/docs/integrations/vectorstores/faiss) 開始，它將 LangChain 相容的內嵌模型(Embedding Model)與 [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) 函式庫(library)整合，使流程在我們的本地機器上快速且可擴展！

**具體來說：**

1.  我們可以透過 `from_texts` 建構函式(function)將我們的對話餵給 [**FAISS 向量存儲庫(Vector Store)**](https://python.langchain.com/docs/integrations/vectorstores/faiss)。這將取得我們的對話資料和內嵌模型(Embedding Model)來創建一個可搜尋的討論索引。

2.  然後這個向量存儲庫(Vector Store)可以被「解釋」為檢索器，支援 LangChain 可運行物件(Runnable) API 並返回透過輸入(Intake)查詢檢索(Retrieval)的文件。

以下顯示了如何使用 LangChain `vectorstore` API 建構 FAISS 向量存儲庫(Vector Store)並將其重新解釋為檢索器：

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 240 ms, sys: 276 ms, total: 516 ms
Wall time: 1.36 s





檢索器現在可以像任何其他 LangChain 可運行物件(Runnable)一樣使用，來查詢向量存儲庫(Vector Store)以獲取一些相關文件：

In [5]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='13857896-6042-4742-981c-91c71ed6a8bc',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='ac0298ae-bbad-4673-8ac5-d4fae1b64aeb',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='5e0cce51-85b1-452f-ab69-b9a3ed3c7de4',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        id='c1ad2694-8c27-4289-b4ed-190814323f38',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [6]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='51cccb61-374e-44be-94da-e03171e5c42d',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='c1ad2694-8c27-4289-b4ed-190814323f38',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='13857896-6042-4742-981c-91c71ed6a8bc',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='ac0298ae-bbad-4673-8ac5-d4fae1b64aeb',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]


<br>


如我們所見，我們的檢索器從我們的查詢中找到了一些語義相關的文件。您可能會注意到並非所有文件都有用或本身清晰。例如，對於*「你的名字」*檢索(Retrieval)*「Beras」*如果脫離脈絡資訊(Context)提供，對聊天機器人來說可能會有問題。預期潛在問題並在您的 LLM 組件之間創建協同(Synergized)可以增加良好 RAG 行為的可能性，所以請留意這些陷阱和機會。

### **步驟 3：** 將對話檢索(Retrieval)整合到我們的鏈(Chain)中


現在我們有了載入的檢索器組件作為鏈(Chain)，我們可以像之前一樣將其整合到我們現有的聊天系統中。具體來說，我們可以從***始終開啟的 RAG 制定***開始，其中：

-   **檢索器預設始終在檢索(Retrieval)脈絡資訊(Context)** 。

-   **生成器在檢索(Retrieval)的脈絡資訊(Context)上行動**。

In [7]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [8]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Based on the context provided, Beras lives in the Arctic. It seems like the Rocky Mountains are a fascinating place
for Beras to learn about, considering the vastly different climate from where they currently reside!



花一點時間嘗試更多invoke並看看新設置的表現如何。無論您選擇哪個模型，以下問題都應該作為有趣的起點。

In [9]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. You can learn 
more about them by doing some online research or watching documentaries if you're interested!

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Hello there! The Rocky Mountains are a stunning range of mountains that stretch across North America. To answer 
your question about their location, they run from northern British Columbia in Canada all the way down to New 
Mexico in the United States. As for their proximity to California, they don't directly border the state. The 
closest parts of the Rocky Mountains to California would be in Nevada or Idaho, which are still quite a distance 
away. I hope that helps! Let's continue exploring more about the Rocky Mountains if you have any other questions.

In [11]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Unfortunately, the context doesn't provide information on Beras' exact location or distance from the Rocky 
Mountains. I can tell you, though, that the Rocky Mountains are a beautiful and majestic range of mountains 
stretching across North America. If you'd like, I can search for more information about their geographical location
and specific details about their range.

<br>



您可能會注意到這個始終開啟的檢索(Retrieval)節點(Node)在迴圈中有一些不錯的效能，因為實際餵給 LLM 的脈絡資訊(Context)保持相對較小。重要的是要實驗內嵌(Embedding)大小、脈絡資訊(Context)限制和模型選項等因素，看看您可以期待什麼樣的行為，以及哪些努力值得採取來改善效能。

### **步驟 4：** 自動對話儲存


現在我們看到我們的向量存儲庫(Vector Store)記憶體單元應該如何運作，我們可以執行最後一個整合，允許我們的對話向我們的對話添加新資料(Entries)：一個為我們呼叫 `add_texts` 方法來更新存儲狀態的可運行物件(Runnable)。

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying ice cream, it's essential to 
remember that it's a vast mountain range spanning over 3,000 miles across North America. There are many activities 
to do there, like hiking, camping, and sightseeing. However, if you find yourself near a local town or village, 
there might be a place to grab some ice cream and savor the flavors as you take in the breathtaking views! But 
Beras, let's focus on the incredible natural wonders the Rocky Mountains have to offer first!

Well, Beras, based on our conversation, it seems your favorite food might just be ice cream! Remember, as much as 
you're excited about enjoying ice cream amidst the stunning Rocky Mountains, don't forget to explore all the 
incredible natural wonders it has to offer first!

I apologize for the misunderstanding, Beras. I inferred that ice cream was your favorite food because it came up in
our discussion about the Rocky Mountains. From what I can see now, I must have misunderstood. I'm glad you've 
clarified that your true favorite is honey! I'll make sure to remember that for future reference.

Absolutely, Beras! I do now. I'm sorry for the confusion earlier. Based on your clarification, I now understand 
that your favorite food is honey. I'll keep that in mind for our future conversations.


與更自動的將脈絡資訊(Context)注入LLM 的全文(full-text)或基於規則(rule-based approaches)的方法不同，這種方法確保了一定程度的整合(Aggregate)，可以防止脈絡資訊(Context)長度失控。它本身仍然不是一個萬無一失的策略，但對於非結構化對話來說是一個顯著的改進（甚至不需要強大的指令調整模型來執行槽填充）。



<br>

## **第三部分 [練習]：** 文件分塊檢索(Retrieval)的 RAG


考慮到我們之前對文件載入的探索，資料分塊可以被內嵌(Embedding)和搜尋的想法可能並不令人驚訝。話雖如此，這絕對值得回顧，因為將 RAG 應用於文件是一把雙刃劍；它可能**看起來**開箱即用運作良好，但在為真正可靠的效能最佳化時需要一些額外的注意。它也提供了一個絕佳的機會來回顧一些基本的 LCEL 技能，所以讓我們看看我們能做什麼！
<br>

### **練習：**


在前面的範例中，您可能記得我們使用 [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) 的幫助拉入了一些相對較小的論文，使用以下語法：

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

考慮到您到目前為止學到的一切，選擇您想要使用的論文選擇，並開發一個可以談論它們的聊天機器人！

雖然這是一個相當大的任務，但下面將提供***大部分***流程的演練。在演練結束時，將提供許多必要的拼圖片段，您的真正任務將是將它們整合在一起用於最終的 `retrieval_chain`。完成後，準備在最後一個 notebook 中重新整合鏈(Chain)（或您選擇的風格）作為評量(Assessment)練習的一部分！

<br>








<br>

### **任務 1**：載入和分塊(Chunking)您的文件


以下程式碼區塊為您提供了一些預設論文，供您的 RAG 鏈(Chain)載入。請隨意選擇更多論文，但請注意較長的文件將需要更長的處理時間。包含了一些簡化假設和額外的處理步驟，以幫助您改善最單純的(naive) RAG 效能：

-   如果存在「References」部分，文件會在此之前被截斷。這將防止我們的系統考慮引用和附錄部分，這些部分往往很長且令人分心。

-   插入一個列出可用文件的分塊，以在單一分塊中提供所有可用文件的高階視圖。如果您的管線(Pipeline)在每次檢索(Retrieval)時不提供中介資料(Metadata)，這是一個有用的組件，甚至可以在適當時列在更高優先級片段的清單中。

-   此外，中介資料(Metadata)資料(Entries)也被插入以提供一般資訊。理想情況下，還會有一些綜合結果(Synthesis)分塊將中介資料(Metadata)合併到有趣的跨文件分塊中。

**注意：** ***為了評量(Assessment)的目的，請至少包含一篇不到一個月的論文！***


In [13]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2022-05-01',
    'Title': 'MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external 
knowledge sources and discrete reasoning',
    'Authors': 'Ehud Karpas, Omri Abend, Yonatan Belinkov, Barak Lenz, Opher Lieber, Nir Ratner, Yoav Shoham, Hofit
Bata, Yoav Levine, Kevin Leyton-Brown, Dor Muhlgay, Noam Rozen, Erez Schwartz, Gal Shachaf, Shai Shalev-Shwartz, 
Amnon Shashua, Moshe Tenenholtz',
    'Summary': 'Huge language models (LMs) have ushered in a new era for AI, serving as a\ngateway to 
natural-language-based knowledge tasks. Although an essential\nelement of modern AI, LMs are also inherently 
limited in a number of ways. We\ndiscuss these limitations and how they can be avoided by adopting a 
systems\napproach. Conceptualizing the challenge as one that involves knowledge and\nreasoning in addition to 
linguistic processing, we define a flexible\narchitecture with multiple neural models, complemented by discrete 
knowledge\nand reasoning modules. We describe this neuro-symbolic architecture, dubbed the\nModular Reasoning, 
Knowledge and Language (MRKL, pronounced "miracle") system,\nsome of the technical challenges in implementing it, 
and Jurassic-X, AI21 Labs\'\nMRKL system implementation.'
}


Document 4
 - # Chunks: 21
 - Metadata: 


{
    'Published': '2023-10-10',
    'Title': 'Mistral 7B',
    'Authors': 'Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, 
Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed',
    'Summary': 'We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered\nfor superior 
performance and efficiency. Mistral 7B outperforms Llama 2 13B\nacross all evaluated benchmarks, and Llama 1 34B in
reasoning, mathematics, and\ncode generation. Our model leverages grouped-query attention (GQA) for 
faster\ninference, coupled with sliding window attention (SWA) to effectively handle\nsequences of arbitrary length
with a reduced inference cost. We also provide a\nmodel fine-tuned to follow instructions, Mistral 7B -- Instruct, 
that surpasses\nthe Llama 2 13B -- Chat model both on human and automated benchmarks. Our\nmodels are released 
under the Apache 2.0 license.'
}


Document 5
 - # Chunks: 44
 - Metadata: 


{
    'Published': '2023-12-24',
    'Title': 'Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena',
    'Authors': 'Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang, Zi Lin, 
Zhuohan Li, Dacheng Li, Eric P. Xing, Hao Zhang, Joseph E. Gonzalez, Ion Stoica',
    'Summary': 'Evaluating large language model (LLM) based chat assistants is challenging\ndue to their broad 
capabilities and the inadequacy of existing benchmarks in\nmeasuring human preferences. To address this, we explore
using strong LLMs as\njudges to evaluate these models on more open-ended questions. We examine the\nusage and 
limitations of LLM-as-a-judge, including position, verbosity, and\nself-enhancement biases, as well as limited 
reasoning ability, and propose\nsolutions to mitigate some of them. We then verify the agreement between 
LLM\njudges and human preferences by introducing two benchmarks: MT-bench, a\nmulti-turn question set; and Chatbot 
Arena, a crowdsourced battle platform. Our\nresults reveal that strong LLM judges like GPT-4 can match both 
controlled and\ncrowdsourced human preferences well, achieving over 80% agreement, the same\nlevel of agreement 
between humans. Hence, LLM-as-a-judge is a scalable and\nexplainable way to approximate human preferences, which 
are otherwise very\nexpensive to obtain. Additionally, we show our benchmark and traditional\nbenchmarks complement
each other by evaluating several variants of LLaMA and\nVicuna. The MT-bench questions, 3K expert votes, and 30K 
conversations with\nhuman preferences are publicly available 
at\nhttps://github.com/lm-sys/FastChat/tree/main/fastchat/llm_judge.'
}

<br>

### **任務 2**：建構您的文件向量存儲庫(Vector Store)


現在我們有了所有組件，我們可以繼續創建圍繞它們的索引：

In [14]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 587 ms, sys: 53.1 ms, total: 640 ms
Wall time: 17.4 s


<br>


從那裡，我們可以使用以下實用工具將我們的索引合併為單一索引：

In [15]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 238 chunks


<br>

### **任務 3：[練習]** 實作您的 RAG 鏈(Chain)


最後，所有拼圖片段都已就位來實作 RAG 管線(Pipeline)！作為回顧，我們現在有：

-   一種為對話記憶(conversation memory)從頭建構向量存儲庫(Vector Store)的方法（以及使用 `default_FAISS()` 初始化空向量存儲庫(Vector Store)的方法）

-   一個預載了來自我們 `ArxivLoader` 實用工具的有用文件資訊的向量存儲庫(Vector Store)（儲存在 `docstore` 中）。

在幾個額外實用工具的幫助下，您終於準備好整合您的鏈(Chain)了！提供了一些額外的便利實用工具（`doc2str` 和現在常見(popular)的 `RPrint`），但使用是可選的。此外，還定義了一些起始提示(Prompt)和結構。

> **考慮到所有這些：** 請實作 `retrieval_chain`。

In [17]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!


retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)

## END TODO
################################################################################################
def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{
    'input': 'Tell me about RAG!',
    'history': '',
    'context': '[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . We refer to this 
decoding procedure as \\u201cThorough Decoding.\\u201d For longer\\noutput sequences, |Y | can become large, 
requiring many forward passes. For more ef\\ufb01cient decoding,\\nwe can make a further approximation that 
p\\u03b8(y|x, zi) \\u22480 where y was not generated during beam\\nsearch from x, zi. This avoids the need to run 
additional forward passes once the candidate set Y has\\nbeen generated. We refer to this decoding procedure as 
\\u201cFast Decoding.\\u201d\\n3\\nExperiments\\nWe experiment with RAG in a wide range of knowledge-intensive 
tasks. For all experiments, we use\\na single Wikipedia dump for our non-parametric knowledge source. Following Lee
et al. [31] and\\nKarpukhin et al. [26], we use the December 2018 dump. Each Wikipedia article is split into 
disjoint\\n100-word chunks, to make a total of 21M documents\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . Since RAG can be\\nemployed as a language model, similar concerns as for GPT-2 
[50] are valid here, although arguably\\nto a lesser extent, including that it might be used to generate abuse, 
faked or misleading content in\\nthe news or on social media; to impersonate others; or to automate the production 
of spam/phishing\\ncontent [54]. Advanced language models may also lead to the automation of various jobs in 
the\\ncoming decades [16]. In order to mitigate these risks, AI systems could be employed to \\ufb01ght 
against\\nmisleading content and automated spam/phishing.\\nAcknowledgments\\nThe authors would like to thank the 
reviewers for their thoughtful and constructive feedback on this\\npaper, as well as HuggingFace for their help in 
open-sourcing code to run RAG models. The authors\\nwould also like to thank Kyunghyun Cho and Sewon Min for 
productive discussions and advice. EP\\nthanks supports from the NSF Graduate Research Fellowship. PL is supported 
by the FAIR PhD\\nprogram.\\n\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] 
.\\nRAG-Token\\nThe RAG-Token model can be seen as a standard, autoregressive seq2seq genera-\\ntor with transition
probability: p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) = P\\nz\\u2208top-k(p(\\u00b7|x)) p\\u03b7(zi|x)p\\u03b8(yi|x, 
zi, y1:i\\u22121) To\\ndecode, we can plug p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) into a standard beam 
decoder.\\nRAG-Sequence\\nFor RAG-Sequence, the likelihood p(y|x) does not break into a conventional per-\\ntoken 
likelihood, hence we cannot solve it with a single beam search. Instead, we run beam search for\\neach document z, 
scoring each hypothesis using p\\u03b8(yi|x, z, y1:i\\u22121). This yields a set of hypotheses\\nY , some of which 
may not have appeared in the beams of all documents. To estimate the probability\\nof an hypothesis y we run an 
additional forward pass for each document z for which y does not\\nappear in the beam, multiply generator 
probability with p\\u03b7(z|x) and then sum the probabilities across\\nbeams for the marginals\n[Quote from 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . In one approach, RAG-Sequence, the model uses 
the same document\\nto predict each target token. The second approach, RAG-Token, can predict each target token 
based\\non a different document. In the following, we formally introduce both models and then describe 
the\\np\\u03b7 and p\\u03b8 components, as well as the training and decoding 
procedure.\\n2.1\\nModels\\nRAG-Sequence Model\\nThe RAG-Sequence model uses the same retrieved document to 
generate\\nthe complete sequence. Technically, it treats the retrieved document as a single latent variable 
that\\nis marginalized to get the seq2seq probability p(y|x) via a top-K approximation\n'
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . We refer to this decoding procedure as \\u201cThorough Decoding.\\u201d For 
longer\\noutput sequences, |Y | can become large, requiring many forward passes. For more ef\\ufb01cient 
decoding,\\nwe can make a further approximation that p\\u03b8(y|x, zi) \\u22480 where y was not generated during 
beam\\nsearch from x, zi. This avoids the need to run additional forward passes once the candidate set Y has\\nbeen
generated. We refer to this decoding procedure as \\u201cFast Decoding.\\u201d\\n3\\nExperiments\\nWe experiment 
with RAG in a wide range of knowledge-intensive tasks. For all experiments, we use\\na single Wikipedia dump for 
our non-parametric knowledge source. Following Lee et al. [31] and\\nKarpukhin et al. [26], we use the December 
2018 dump. Each Wikipedia article is split into disjoint\\n100-word chunks, to make a total of 21M 
documents\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . Since RAG can 
be\\nemployed as a language model, similar concerns as for GPT-2 [50] are valid here, although arguably\\nto a 
lesser extent, including that it might be used to generate abuse, faked or misleading content in\\nthe news or on 
social media; to impersonate others; or to automate the production of spam/phishing\\ncontent [54]. Advanced 
language models may also lead to the automation of various jobs in the\\ncoming decades [16]. In order to mitigate 
these risks, AI systems could be employed to \\ufb01ght against\\nmisleading content and automated 
spam/phishing.\\nAcknowledgments\\nThe authors would like to thank the reviewers for their thoughtful and 
constructive feedback on this\\npaper, as well as HuggingFace for their help in open-sourcing code to run RAG 
models. The authors\\nwould also like to thank Kyunghyun Cho and Sewon Min for productive discussions and advice. 
EP\\nthanks supports from the NSF Graduate Research Fellowship. PL is supported by the FAIR 
PhD\\nprogram.\\n\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] 
.\\nRAG-Token\\nThe RAG-Token model can be seen as a standard, autoregressive seq2seq genera-\\ntor with transition
probability: p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) = P\\nz\\u2208top-k(p(\\u00b7|x)) p\\u03b7(zi|x)p\\u03b8(yi|x, 
zi, y1:i\\u22121) To\\ndecode, we can plug p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) into a standard beam 
decoder.\\nRAG-Sequence\\nFor RAG-Sequence, the likelihood p(y|x) does not break into a conventional per-\\ntoken 
likelihood, hence we cannot solve it with a single beam search. Instead, we run beam search for\\neach document z, 
scoring each hypothesis using p\\u03b8(yi|x, z, y1:i\\u22121). This yields a set of hypotheses\\nY , some of which 
may not have appeared in the beams of all documents. To estimate the probability\\nof an hypothesis y we run an 
additional forward pass for each document z for which y does not\\nappear in the beam, multiply generator 
probability with p\\u03b7(z|x) and then sum the probabilities across\\nbeams for the marginals\n[Quote from 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . In one approach, RAG-Sequence, the model uses 
the same document\\nto predict each target token. The second approach, RAG-Token, can predict each target token 
based\\non a different document. In the following, we formally introduce both models and then describe 
the\\np\\u03b7 and p\\u03b8 components, as well as the training and decoding 
procedure.\\n2.1\\nModels\\nRAG-Sequence Model\\nThe RAG-Sequence model uses the same retrieved document to 
generate\\nthe c

RAG stands for Retrieval-Augmented Generation. It is a method that combines a retriever model, which identifies relevant documents from a large corpus, with a generator model, which produces answers to natural language processing (NLP) tasks. The retriever model is based on the DPR (Dense Passage Retrieval) model, and the generator model is based on a transformer model.

RAG can be employed in a variety of knowledge-intensive NLP tasks and is trained on a single Wikipedia dump, which is split into disjoint 100-word chunks to make a total of 21M documents.

There are two ways that RAG can be used, RAG-Sequence and RAG-Token. RAG-Sequence uses the same document to predict each target token, while RAG-Token can predict each target token based on a different document.

There are also two decoding procedures that can be used with RAG, Thorough Decoding and Fast Decoding. Thorough decoding runs additional forward passes for longer output sequences, while Fast Decoding makes an approximation 

### **任務 4：** 與您的 Gradio 聊天機器人互動

In [ ]:
# chatbot = gr.Chatbot(value = [[None, initial_msg]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

# try:
#     demo.launch(debug=True, share=True, show_api=False)
#     demo.close()
# except Exception as e:
#     demo.close()
#     print(e)
#     raise e

<br>



<br>

## **第四部分：** 為評量(Assessment)保存您的索引


在您實作了 RAG 鏈(Chain)之後，請如 [官方文件](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading) 所示保存您累積的向量存儲庫(Vector Store)。您將有機會在最終評量(Assessment)中再次使用它！

In [18]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss



如果一切都正確保存，可以invoke以下行從壓縮的 `tgz` 檔案中拉取索引（假設已安裝 pip 要求）。在您確認程式碼區塊(Cell)可以拉入您的索引後，下載 `docstore_index.tgz` 以在最後一個 notebook 中使用！

In [19]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
. To demonstrate, we build an index using the DrQA [5]\nWikipedia dump from December 2016 and compare outputs from RAG using this index to the newer\nindex from our main results (December 2018). We prepare a list of 82 world leaders who had changed\n7\nTable 4: Human assessments for the Jeopardy\nQuestion Generation Task.\nFactuality\nSpeci\ufb01city\nBART better\n7.1%\n16.8%\nRAG better\n42.7%\n37.4%\nBoth good\n11.7%\n11.8%\nBoth poor\n17.7%\n6.9%\nNo majority\n20.8%\n20.1%\nTable 5: Ratio of distinct to total tri-grams for\ngeneration tasks.\nMSMARCO\nJeopardy QGen\nGold\n89.6%\n90.0%\nBART\n70.7%\n32.4%\nRAG-Token\n77.8%\n46.8%\nRAG-Seq.\n83.5%\n53.8%\nTable 6: Ablations on the dev set. As FEVER is a classi\ufb01cation task, both RAG models are equivalent.\nModel\nNQ\nTQA\nWQ\nCT\nJeopardy-QGen\nMSMarco\nFVR-3\nFVR-2\nExact Match\nB-1\nQB-1\nR-L\nB-1\nLabel Accuracy\nRAG-Token-BM25\n29.7\n41.5\n32.1\n33.1\n17.5\n22


<br>

## **第五部分：** 總結


恭喜！假設您的 RAG 鏈(Chain)一切順利，您現在準備好進入 **RAG 評量(Assessment) [評量(Assessment)]** 部分！

<font color="#76b900">**做得很好！**</font>

### **下一步：**


1.  **[可選]** 重新訪問(Navigate) notebook 頂部的**「值得思考的問題」部分**，並思考一些可能的答案。

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

